In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 
import gc
import time
import torch
torch.cuda.empty_cache()
import torch.nn as nn
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import RobertaTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification, EarlyStoppingCallback
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

def compute_metrics(p: transformers.EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {"eval_accuracy": accuracy_score(p.label_ids, preds)}

class SentimentDataset(Dataset):
    def __init__(self, input_list, labels, tokenizer):
        self.encoded_data = tokenizer(input_list, 
                                      truncation=True, 
                                      padding=True, 
                                      return_tensors="pt", 
                                      max_length=256)
       
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        item = {
            "input_ids": self.encoded_data['input_ids'][idx],
            "attention_mask": self.encoded_data['attention_mask'][idx],
            "labels": self.labels[idx],
        }
        return item

    def __len__(self):
        return len(self.labels)

class FineTuner:
    def __init__(self, 
                 model_name, 
                 output_dir,
                 logging_dir):
        self.tokenizer = RobertaTokenizer.from_pretrained(model_name)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.base_model = AutoModelForSequenceClassification.from_pretrained(model_name, output_attentions=False).to(self.device)
        self.training_args = None
        self.train_dataset = None
        self.val_dataset = None
        self.output_dir = output_dir
        self.logging_dir = logging_dir

    # Usage within some class or function:
    def prepare_data(self, train_input_list, train_label, val_input_list, val_label):
        self.train_dataset = SentimentDataset(train_input_list, train_label, self.tokenizer)
        self.val_dataset = SentimentDataset(val_input_list, val_label, self.tokenizer)

    def setup_training(self, batch_size=64, num_epochs=5):
        self.training_args = TrainingArguments(
            output_dir=self.output_dir,
            num_train_epochs=num_epochs,
            logging_dir=self.logging_dir,
            logging_steps=1000,  # Log every 100 steps
            evaluation_strategy="epoch",  # Evaluate after each epoch
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            learning_rate=2e-5,
            weight_decay=0.01,
            warmup_steps=500, 
            save_strategy="epoch",  # Do not save any models except the best one
            load_best_model_at_end=True,  # Load the best model in terms of evaluation metric
            metric_for_best_model="eval_accuracy",  # Define the desired metric here
            greater_is_better=True,
            gradient_accumulation_steps=4,
            max_grad_norm=1.0,
        )
            
    def fine_tune_all(self):
        trainer = Trainer(
            model=self.base_model,
            args=self.training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.val_dataset,
            compute_metrics=compute_metrics,  # Add this line
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )
        trainer.train()

In [5]:
from datasets import load_dataset

# Load IMDb dataset
dataset = load_dataset("imdb")

# Prepare 100 samples for training and 20 samples for testing
train_data = dataset["train"].shuffle(seed=42).select(range(100))
val_data = dataset["train"].shuffle(seed=42).select(range(10))
test_data = dataset["test"].shuffle(seed=42).select(range(20))

# Organize as train_x (sentences) and train_y (labels)
train_x = train_data["text"]
train_y = train_data["label"]

val_x = val_data["text"]
val_y = val_data["label"]

# Organize as test_x (sentences) and test_y (labels)
test_x = test_data["text"]
test_y = test_data["label"]

Using the latest cached version of the module from /home/lei/.cache/huggingface/modules/datasets_modules/datasets/imdb/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0 (last modified on Sun Jun 11 15:07:30 2023) since it couldn't be found locally at imdb, or remotely on the Hugging Face Hub.


In [6]:

tuner = FineTuner(model_name="siebert/sentiment-roberta-large-english",
                    output_dir = '../results/roberta',
                    logging_dir = '../logs/roberta')

tuner.prepare_data(train_x, train_y, val_x, val_y)
tuner.setup_training(batch_size=64)
tuner.fine_tune_all()

/home/lei/anaconda3/envs/RLF/lib/python3.8/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.702061,0.900000
2,No log,0.701827,0.900000
3,No log,0.701158,0.900000
